# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




In [1]:
# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Tue May  4 17:45:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !ps
# !kill 1754

# mount google drive

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [4]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 67.5MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [5]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

## Import Packages

In [6]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True


In [7]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer




 

In [8]:
# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# model = BertForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext").to(device)
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-roberta-wwm-ext")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [9]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [10]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

for i in range(len(train_questions)):
  train_questions_tokenized[i].raw = train_questions[i]["question_text"]

for i in range(len(dev_questions)):
  dev_questions_tokenized[i].raw = dev_questions[i]["question_text"]

for i in range(len(test_questions)):
  test_questions_tokenized[i].raw = test_questions[i]["question_text"]

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

for i in range(len(train_paragraphs)):
  train_paragraphs_tokenized[i].raw = train_paragraphs[i]

for i in range(len(dev_paragraphs)):
  dev_paragraphs_tokenized[i].raw = dev_paragraphs[i]

for i in range(len(test_paragraphs)):
  test_paragraphs_tokenized[i].raw = test_paragraphs[i]


# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [11]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 60
        self.max_paragraph_len = 449
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 50

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            ans_len = answer_end_token - answer_start_token + 1
            paragraph_start = answer_end_token - random.randint(ans_len-1, self.max_paragraph_len-1)
            paragraph_start = max(0, paragraph_start)
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list, is_list = [], [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
                is_list.append(i)

            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list), torch.tensor(is_list), question["paragraph_id"], len(input_ids_question)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 16

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)


## Function for Evaluation

In [12]:
def evaluate(data, output, tokenized_paragraphs, tokenized_questions):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    tokenized_paragraph = tokenized_paragraphs[data[4]]
    tokenized_question = tokenized_questions[data[4]]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob and end_index+1 > start_index:

            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            token_to_chars = lambda idx: tokenized_paragraph.token_to_chars(data[3][0][k].item() + idx - data[5][0].item())
            get_chars = lambda idx: tokenized_paragraph.raw[token_to_chars(idx)[0]: token_to_chars(idx)[1]]

            answer = ''
            for i in range(start_index, end_index+1):
              idx = data[3][0][k].item() + i - data[5][0].item()

              if idx >= 0:
                char_idxs = tokenized_paragraph.token_to_chars(idx)
                answer += tokenized_paragraph.raw[char_idxs[0]: char_idxs[1]]                
            
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [13]:
def train(model, train_loader, dev_loader):
    num_epoch = 10
    validation = False
    logging_step = 100
    learning_rate = 1e-4
    tot_step = 4000
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    if fp16_training:
        model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

    model.train()

    print("Start Training ...")

    step = 1
    for epoch in range(num_epoch):
        if step >= tot_step:
            break

        train_loss = train_acc = 0
        
        for data in tqdm(train_loader):	
            if step >= tot_step:
                break

            # Load all data into GPU
            data = [i.to(device) for i in data]
            
            # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
            # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
            output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

            # Choose the most probable start position / end position
            start_index = torch.argmax(output.start_logits, dim=1)
            end_index = torch.argmax(output.end_logits, dim=1)
            
            # Prediction is correct only if both start_index and end_index are correct
            train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
            train_loss += output.loss
            
            if fp16_training:
                accelerator.backward(output.loss)
            else:
                output.loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            step += 1

            ##### TODO: Apply linear learning rate decay #####
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate - learning_rate * step / tot_step
                param_group['lr'] = max(0, param_group['lr'])
              
            # Print training loss and accuracy over past logging step
            if step % logging_step == 0:
                print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
                print(f"learning rate: {param_group['lr']}")
                train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                        attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                tokenized_paragraphs = dev_loader.dataset.tokenized_paragraphs
                tokenized_questions = dev_loader.dataset.tokenized_questions
                dev_acc += evaluate(data, output, tokenized_paragraphs, tokenized_questions) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

    # Save a model and its configuration file to the directory 「saved_model」 
    # i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
    # Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
    print("Saving Model ...")
    model_save_dir = "saved_model" 
    model.save_pretrained(model_save_dir)

## Testing

In [14]:
def predict(model, test_loader, seed):
    print("Evaluating Test Set ...")

    result = []

    model.eval()
    with torch.no_grad():
        for data in tqdm(test_loader):
            output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                          attention_mask=data[2].squeeze(dim=0).to(device))
            
            tokenized_paragraphs = test_loader.dataset.tokenized_paragraphs
            tokenized_questions = test_loader.dataset.tokenized_questions
            result.append(evaluate(data, output, tokenized_paragraphs, tokenized_questions))

    result_file = f"gdrive/MyDrive/ColabNotebooks/HW7/data/result_{seed}.csv"
    with open(result_file, 'w') as f:
        f.write("ID,Answer\n")
        for i, test_question in enumerate(test_questions):
            # Replace commas in answers with empty strings (since csv is separated by comma)
            # Answers in kaggle are processed in the same way
            f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

    print(f"Completed! Result is in {result_file}")

# Ensemble

In [15]:
# same_seeds(17)
# model = BertForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext").to(device)
# train(model, train_loader, dev_loader)
# predict(model, test_loader, 17)

# from 12
for i in range(12, 20): 
    print(f'round {i}')
    same_seeds(i)
    model = BertForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext").to(device)
    train(model, train_loader, dev_loader)
    predict(model, test_loader, i)
    

round 18


Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chine

Start Training ...


Epoch 1 | Step 100 | loss = 2.000, acc = 0.407
learning rate: 9.75e-05
Epoch 1 | Step 200 | loss = 0.978, acc = 0.629
learning rate: 9.5e-05
Epoch 1 | Step 300 | loss = 0.929, acc = 0.651
learning rate: 9.25e-05
Epoch 1 | Step 400 | loss = 0.811, acc = 0.673
learning rate: 9e-05
Epoch 1 | Step 500 | loss = 0.823, acc = 0.699
learning rate: 8.75e-05
Epoch 1 | Step 600 | loss = 0.762, acc = 0.678
learning rate: 8.5e-05
Epoch 1 | Step 700 | loss = 0.716, acc = 0.720
learning rate: 8.25e-05
Epoch 1 | Step 800 | loss = 0.723, acc = 0.710
learning rate: 8e-05
Epoch 1 | Step 900 | loss = 0.730, acc = 0.719
learning rate: 7.75e-05
Epoch 1 | Step 1000 | loss = 0.708, acc = 0.719
learning rate: 7.500000000000001e-05
Epoch 1 | Step 1100 | loss = 0.680, acc = 0.725
learning rate: 7.25e-05
Epoch 1 | Step 1200 | loss = 0.668, acc = 0.721
learning rate: 7.000000000000001e-05
Epoch 1 | Step 1300 | loss = 0.640, acc = 0.736
learning rate: 6.75e-05
Epoch 1 | Step 1400 | loss = 0.677, acc = 0.722
learnin

Epoch 2 | Step 1700 | loss = 0.059, acc = 0.125
learning rate: 5.75e-05
Epoch 2 | Step 1800 | loss = 0.357, acc = 0.833
learning rate: 5.5e-05
Epoch 2 | Step 1900 | loss = 0.355, acc = 0.819
learning rate: 5.25e-05
Epoch 2 | Step 2000 | loss = 0.352, acc = 0.829
learning rate: 5e-05
Epoch 2 | Step 2100 | loss = 0.366, acc = 0.835
learning rate: 4.75e-05
Epoch 2 | Step 2200 | loss = 0.322, acc = 0.844
learning rate: 4.5e-05
Epoch 2 | Step 2300 | loss = 0.315, acc = 0.851
learning rate: 4.25e-05
Epoch 2 | Step 2400 | loss = 0.360, acc = 0.845
learning rate: 4e-05
Epoch 2 | Step 2500 | loss = 0.334, acc = 0.840
learning rate: 3.7500000000000003e-05
Epoch 2 | Step 2600 | loss = 0.329, acc = 0.848
learning rate: 3.5e-05
Epoch 2 | Step 2700 | loss = 0.331, acc = 0.847
learning rate: 3.2500000000000004e-05
Epoch 2 | Step 2800 | loss = 0.358, acc = 0.841
learning rate: 2.9999999999999997e-05
Epoch 2 | Step 2900 | loss = 0.321, acc = 0.845
learning rate: 2.749999999999999e-05
Epoch 2 | Step 300

...

Epoch 3 | Step 3400 | loss = 0.052, acc = 0.279
learning rate: 1.4999999999999999e-05
Epoch 3 | Step 3500 | loss = 0.143, acc = 0.914
learning rate: 1.2499999999999992e-05
Epoch 3 | Step 3600 | loss = 0.144, acc = 0.923
learning rate: 9.999999999999999e-06
Epoch 3 | Step 3700 | loss = 0.166, acc = 0.915
learning rate: 7.500000000000006e-06
Epoch 3 | Step 3800 | loss = 0.166, acc = 0.918
learning rate: 4.9999999999999996e-06
Epoch 3 | Step 3900 | loss = 0.134, acc = 0.930
learning rate: 2.5000000000000066e-06
Epoch 3 | Step 4000 | loss = 0.156, acc = 0.919
learning rate: 0
Saving Model ...
Evaluating Test Set ...



Completed! Result is in gdrive/MyDrive/ColabNotebooks/HW7/data/result_19.csv
round 20


Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chine

Start Training ...


RuntimeError: ignored

# majority vote

In [16]:
import csv
import json

rowss = []
for i in range(12, 20):
    with open(f'gdrive/MyDrive/ColabNotebooks/HW7/data/result_{i}.csv') as f:
        rows = list(csv.reader(f))
        if len(rowss) == 0:
            for j in range(len(rows)):
                rowss.append([]) 
        
        for j in range(len(rows)):
            rowss[j].append(json.dumps(rows[j]))

rows_majority = []
for rows in rowss:
    table = {}
    for row in rows:
        if row not in table:
            table[row] = 0
        
        table[row] += 1
    
    row_majority = json.loads(max(table, key=lambda row: table[row]))
    rows_majority.append(row_majority)

with open('gdrive/MyDrive/ColabNotebooks/HW7/data/result_majority.csv', 'w') as f:
    writer = csv.writer(f)
    for row in rows_majority:
        writer.writerow(row)


with open('./result_majority.csv', 'w') as f:
    writer = csv.writer(f)
    for row in rows_majority:
        writer.writerow(row)
